In [2]:
import json 
import pandas as pd
import geopandas as gpd

import glob                             #verwendung von 'wildacards' in Dateinamen
from tqdm import tqdm                   #fuer Darstellen eines Fortschrittsbalkens
from datetime import date
import matplotlib.pyplot as plt

#import seaborn as sns 

In [3]:
'''Einlesen der Daten 

Erwartet dass unter OBS-roh eine Datei (geojson) liegt, die vorher mit 1-1_get-newest-OBS erstellt wurde
Erwartet dass unter OSM-roh eine Datei (gpkg) liegt, die vorher mit 1-0_OSM-aufbereiten erstellt wurde'''

#def OSM2OBS(path_OSM, path_OBS):        #erwartet Pfad des Unterordners an dem OSM-Datei als *.gpkg liegt und OBS als *.geojson

#Pfade erstellen (für Verwendung in Funktion jeweils erste Zeile weglassen)
path_OSM = '../OSM-roh/230315'
path_OSM = path_OSM + '/*.gpkg'

#liest alle (sollte aber nur eine sein!) Dateien aus dem Verzeichnis mit osm Dateien (Format gpkg)
files_OSM = glob.glob(path_OSM)
files_OSM

['../OSM-roh/230315/drivable_infra_Brandenburg_230315.gpkg']

In [4]:
#liest die gpkg mit osm daten ein, hier aber nur die linien.
osm_netz=gpd.read_file(files_OSM[0], layer='lines')   

# plot every 100th way
#osm_netz[::100].plot(figsize=(10,10))

In [122]:
# Sicherung eines importierten Netzes, wenn ein zweites Netz importiert werden soll
osm_netzBack = osm_netz  
files_OSMBack = files_OSM

In [127]:
# Variablentausch, wenn zwei Netze importiert sind.
osm_netzTemp = osm_netzBack
osm_netzBack = osm_netz
osm_netz = osm_netzTemp

files_OSMTemp = files_OSMBack
files_OSMBack = files_OSM
files_OSM = files_OSMTemp

In [5]:
'''
Anzahl Radstreifen und davon mit: 
- Breite Radweg
- Breite Fahrbahn
- Breite Fahrbahn & Radweg
- Info zum Parken ()  ---> nicht ausgewertet, da sehr aufwändig herauszufinden
- Infos zu Buffer, davon Buffer = no
- Infos zu Linientyp links
'''
#-------------------------Radstreifen-----------------------------
#Straßen, die ein oder beidseitig Radstreifen haben
osm_lane = osm_netz[(osm_netz['cycleway'] == 'lane') | 
                    (osm_netz['cycleway_both'] == 'lane') | 
                    (osm_netz['cycleway_left'] == 'lane') | 
                    (osm_netz['cycleway_right'] == 'lane')].copy()
#Bei Straßen mit Radstreifen: gibt es Angaben zur Breite, d.h. es ist ein numerischer Wert eingetragen.
osm_cwidth = osm_lane[(osm_lane['cycleway_width'].isna() == False) | 
                    (osm_lane['cycleway_both_width'].isna() == False) | 
                    (osm_lane['cycleway_right_width'].isna() == False) | 
                    (osm_lane['cycleway_left_width'].isna() == False)].copy()
#Angabe zu linksseitiger Radwegabtrennung
osm_sep = osm_lane[(osm_lane['cycleway_separation_left'].isna() == False) |
                   (osm_lane['cycleway_right_separation_left'].isna() == False) |
                   (osm_lane['cycleway_left_separation_left'].isna() == False) |
                   (osm_lane['cycleway_both_separation_left'].isna() == False) |
                   (osm_lane['cycleway_right_separation_both'].isna() == False) |
                   (osm_lane['cycleway_left_separation_both'].isna() == False) |
                   (osm_lane['cycleway_both_separation_both'].isna() == False) |
                   (osm_lane['separation_left'].isna() == False) |
                   (osm_lane['cycleway_right_marking_left'].isna() == False) |
                   (osm_lane['cycleway_left_marking_left'].isna() == False) |
                   (osm_lane['cycleway_right_marking_both'].isna() == False) |
                   (osm_lane['cycleway_left_marking_both'].isna() == False) |
                   (osm_lane['cycleway_both_marking_left'].isna() == False) |
                   (osm_lane['cycleway_marking_left'].isna() == False) |
                   (osm_lane['cycleway_marking'].isna() == False) |
                   (osm_lane['cycleway_right_marking'].isna() == False) |
                   (osm_lane['cycleway_left_marking'].isna() == False)].copy()
#Angabe zu linksseitiger Radwegabtrennung ist no
osm_sepNo = osm_sep[(osm_sep['cycleway_separation_left'] == 'no') |
                    (osm_sep['cycleway_right_separation_left'] == 'no') |
                    (osm_sep['cycleway_left_separation_left'] == 'no') |
                    (osm_sep['cycleway_both_separation_left'] == 'no') |
                    (osm_sep['cycleway_right_separation_both'] == 'no') |
                    (osm_sep['cycleway_left_separation_both'] == 'no') |
                    (osm_sep['cycleway_both_separation_both'] == 'no') |
                    (osm_sep['separation_left'] == 'no')|
                    (osm_sep['cycleway_right_marking_left'] == 'no') |
                    (osm_sep['cycleway_left_marking_left'] == 'no') |
                    (osm_sep['cycleway_right_marking_both'] == 'no') |
                    (osm_sep['cycleway_left_marking_both'] == 'no') |
                    (osm_sep['cycleway_both_marking_left'] == 'no') |
                    (osm_sep['cycleway_marking_left'] == 'no') |
                    (osm_sep['cycleway_marking'] == 'no') |
                    (osm_sep['cycleway_right_marking'] == 'no') |
                    (osm_sep['cycleway_left_marking'] == 'no')].copy()
#Angabe zu Buffer rechts des Radstreifens
osm_buf = osm_lane[(osm_lane['cycleway_right_buffer_right'].isna() == False) |
                   (osm_lane['cycleway_left_buffer_right'].isna() == False) |
                   (osm_lane['cycleway_both_buffer_right'].isna() == False) |
                   (osm_lane['cycleway_right_buffer_both'].isna() == False) |
                   (osm_lane['cycleway_left_buffer_both'].isna() == False) |
                   (osm_lane['cycleway_both_buffer_both'].isna() == False)].copy()
#Angabe zu Buffer rechts des Radstreifens ist no
osm_bufNo = osm_buf[(osm_buf['cycleway_right_buffer_right'] == 'no') |
                    (osm_buf['cycleway_left_buffer_right'] == 'no') |
                    (osm_buf['cycleway_both_buffer_right'] == 'no') |
                    (osm_buf['cycleway_right_buffer_both'] == 'no') |
                    (osm_buf['cycleway_left_buffer_both'] == 'no') |
                    (osm_buf['cycleway_both_buffer_both'] == 'no')].copy()

#-------------------------Radweg-----------------------------
#Straßen, die ein oder beidseitig Radstreifen haben
osm_track = osm_netz[(osm_netz['cycleway'] == 'track') | 
                    (osm_netz['cycleway_both'] == 'track') | 
                    (osm_netz['cycleway_left'] == 'track') | 
                    (osm_netz['cycleway_right'] == 'track')].copy()
#Bei Straßen mit Radstreifen: gibt es Angaben zur Breite, d.h. es ist ein numerischer Wert eingetragen.
osm_cTwidth = osm_track[(osm_track['cycleway_width'].isna() == False) | 
                    (osm_track['cycleway_both_width'].isna() == False) | 
                    (osm_track['cycleway_right_width'].isna() == False) | 
                    (osm_track['cycleway_left_width'].isna() == False)].copy()
#Angabe zu linksseitiger Radwegabtrennung
osm_Tsep = osm_track[(osm_track['cycleway_separation_left'].isna() == False) |
                   (osm_track['cycleway_right_separation_left'].isna() == False) |
                   (osm_track['cycleway_left_separation_left'].isna() == False) |
                   (osm_track['cycleway_both_separation_left'].isna() == False) |
                   (osm_track['cycleway_right_separation_both'].isna() == False) |
                   (osm_track['cycleway_left_separation_both'].isna() == False) |
                   (osm_track['cycleway_both_separation_both'].isna() == False) |
                   (osm_track['separation_left'].isna() == False) |
                   (osm_track['cycleway_right_marking_left'].isna() == False) |
                   (osm_track['cycleway_left_marking_left'].isna() == False) |
                   (osm_track['cycleway_right_marking_both'].isna() == False) |
                   (osm_track['cycleway_left_marking_both'].isna() == False) |
                   (osm_track['cycleway_both_marking_left'].isna() == False) |
                   (osm_track['cycleway_marking_left'].isna() == False) |
                   (osm_track['cycleway_marking'].isna() == False) |
                   (osm_track['cycleway_right_marking'].isna() == False) |
                   (osm_track['cycleway_left_marking'].isna() == False)].copy()
#Angabe zu linksseitiger Radwegabtrennung ist no
osm_TsepNo = osm_Tsep[(osm_Tsep['cycleway_separation_left'] == 'no') |
                    (osm_Tsep['cycleway_right_separation_left'] == 'no') |
                    (osm_Tsep['cycleway_left_separation_left'] == 'no') |
                    (osm_Tsep['cycleway_both_separation_left'] == 'no') |
                    (osm_Tsep['cycleway_right_separation_both'] == 'no') |
                    (osm_Tsep['cycleway_left_separation_both'] == 'no') |
                    (osm_Tsep['cycleway_both_separation_both'] == 'no') |
                    (osm_Tsep['separation_left'] == 'no')|
                    (osm_Tsep['cycleway_right_marking_left'] == 'no') |
                    (osm_Tsep['cycleway_left_marking_left'] == 'no') |
                    (osm_Tsep['cycleway_right_marking_both'] == 'no') |
                    (osm_Tsep['cycleway_left_marking_both'] == 'no') |
                    (osm_Tsep['cycleway_both_marking_left'] == 'no') |
                    (osm_Tsep['cycleway_marking_left'] == 'no') |
                    (osm_Tsep['cycleway_marking'] == 'no') |
                    (osm_Tsep['cycleway_right_marking'] == 'no') |
                    (osm_Tsep['cycleway_left_marking'] == 'no')].copy()
#Angabe zu Buffer rechts des Radstreifens
osm_Tbuf = osm_track[(osm_track['cycleway_right_buffer_right'].isna() == False) |
                   (osm_track['cycleway_left_buffer_right'].isna() == False) |
                   (osm_track['cycleway_both_buffer_right'].isna() == False) |
                   (osm_track['cycleway_right_buffer_both'].isna() == False) |
                   (osm_track['cycleway_left_buffer_both'].isna() == False) |
                   (osm_track['cycleway_both_buffer_both'].isna() == False)].copy()
#Angabe zu Buffer rechts des Radstreifens ist no
osm_TbufNo = osm_Tbuf[(osm_Tbuf['cycleway_right_buffer_right'] == 'no') |
                    (osm_Tbuf['cycleway_left_buffer_right'] == 'no') |
                    (osm_Tbuf['cycleway_both_buffer_right'] == 'no') |
                    (osm_Tbuf['cycleway_right_buffer_both'] == 'no') |
                    (osm_Tbuf['cycleway_left_buffer_both'] == 'no') |
                    (osm_Tbuf['cycleway_both_buffer_both'] == 'no')].copy()

#-------------------------Separat getagged-----------------------------
#Straßen, die ein oder beidseitig Radstreifen haben
osm_separate = osm_netz[(osm_netz['cycleway'] == 'separate') | 
                    (osm_netz['cycleway_both'] == 'separate') | 
                    (osm_netz['cycleway_left'] == 'separate') | 
                    (osm_netz['cycleway_right'] == 'separate')].copy()


print("Anzahl ways in Datei: " + str(files_OSM) + ": " + str(len(osm_netz)) +
      "\n\nAnzahl ways mit cycleway|cycleway_both|cycleway_left|cycleway_right = lane: " +
    str(len(osm_lane)) + 
      "\nDavon mit:\n- Angabe Straßenbreite: " + 
      str((osm_lane['width'].isna() == False).sum()) + 
     "\n- Angabe Radwegbreite: " +
      str(len(osm_cwidth)) + 
     "\n- Angabe Abtrennung oder Markierung nach links: " +
      str(len(osm_sep)) + 
     "\n--- davon 'no': " +
      str(len(osm_sepNo)) + 
     "\n- Angabe Buffer nach Rechts: " +
      str(len(osm_buf)) + 
     "\n--- davon 'no': " +
      str(len(osm_bufNo)) +
      "\n\nAnzahl ways mit cycleway|cycleway_both|cycleway_left|cycleway_right = track: " +
    str(len(osm_track)) + 
      "\nDavon mit:\n- Angabe Straßenbreite: " + 
      str((osm_track['width'].isna() == False).sum()) + 
     "\n- Angabe Radwegbreite: " +
      str(len(osm_cTwidth)) + 
     "\n- Angabe Abtrennung oder Markierung nach links: " +
      str(len(osm_Tsep)) + 
     "\n--- davon 'no': " +
      str(len(osm_TsepNo)) + 
     "\n- Angabe Buffer nach Rechts: " +
      str(len(osm_Tbuf)) + 
     "\n--- davon 'no': " +
      str(len(osm_TbufNo))+
      "\n\nAnzahl ways mit cycleway|cycleway_both|cycleway_left|cycleway_right = separate: " +
    str(len(osm_separate))
     )

Anzahl ways in Datei: ['../OSM-roh/230315/drivable_infra_Brandenburg_230315.gpkg']: 814693

Anzahl ways mit cycleway|cycleway_both|cycleway_left|cycleway_right = lane: 6344
Davon mit:
- Angabe Straßenbreite: 367
- Angabe Radwegbreite: 625
- Angabe Abtrennung oder Markierung nach links: 854
--- davon 'no': 19
- Angabe Buffer nach Rechts: 127
--- davon 'no': 46

Anzahl ways mit cycleway|cycleway_both|cycleway_left|cycleway_right = track: 8383
Davon mit:
- Angabe Straßenbreite: 333
- Angabe Radwegbreite: 121
- Angabe Abtrennung oder Markierung nach links: 160
--- davon 'no': 2
- Angabe Buffer nach Rechts: 18
--- davon 'no': 6

Anzahl ways mit cycleway|cycleway_both|cycleway_left|cycleway_right = separate: 9699


In [116]:
pd.set_option("display.max_rows", None)
osm_lane.groupby(['cycleway_both_marking_left', 'osm_id']).count()

name  highway  ...  other_tags  geometry
cycleway_both_marking_left osm_id                     ...                      
dashed_line                1056832158     1        1  ...           1         1
                           1056832599     1        1  ...           1         1
                           1064850760     1        1  ...           1         1
                           10654946       1        1  ...           1         1
                           1110738788     1        1  ...           1         1
                           1134775998     1        1  ...           1         1
                           1134775999     1        1  ...           1         1
                           1169045891     1        1  ...           1         1
                           1169045893     1        1  ...           1         1
                           1169809154     1        1  ...           1         1
                           1170527158     1        1  ...           1         1
                           1181719852     1        1  ...           1         1
                           1181757427     1        1  ...           0         1
                           1181757428     1        1  ...           0         1
                           1182279660     1        1  ...           1         1
                           1183339560     1        1  ...           1         1
                           1183348243     1        1  ...           1         1
                           1183348244     1        1  ...           1         1
                           1184300440     1        1  ...           1         1
                           1225509070     1        1  ...           1         1
                           124804737      1        1  ...           1         1
                           128572896      1        1  ...           1         1
                           129138633      1        1  ...           1         1
                           129138634      1        1  ...           1         1
                           129138636      1        1  ...           1         1
                           129138637      1        1  ...           1         1
                           129144756      1        1  ...           1         1
                           141561211      1        1  ...           1         1
                           189217698      1        1  ...           1         1
                           217114842      1        1  ...           1         1
                           220253855      1        1  ...           1         1
                           220883650      1        1  ...           1         1
                           220883651      1        1  ...           1         1
                           223247333      1        1  ...           1         1
                           225079481      1        1  ...           1         1
                           225079641      1        1  ...           1         1
                           23054175       1        1  ...           1         1
                           23054176       1        1  ...           1         1
                           23054178       1        1  ...           1         1
                           232836130      0        1  ...           1         1
                           232836139      1        1  ...           0         1
                           23414988       1        1  ...           1         1
                           23511065       1        1  ...           1         1
                           25306823       1        1  ...           1         1
                           260037651      1        1  ...           0         1
                           293869434      1        1  ...           1         1
                           294026101      1        1  ...           1         1
                           294999169      1        1  ...           1         1
                           30298343    